<a href="https://colab.research.google.com/github/jankovicsandras/plpgsql_bm25/blob/main/plpgsql_bm25_comparison_with_paradedb_pg_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Notes
https://github.com/paradedb/paradedb/issues/1631
https://github.com/paradedb/paradedb/issues/1567
https://github.com/paradedb/paradedb/issues/1410
https://github.com/paradedb/paradedb/issues/1289
"""

'\nNotes\nhttps://github.com/paradedb/paradedb/issues/1631\nhttps://github.com/paradedb/paradedb/issues/1567\nhttps://github.com/paradedb/paradedb/issues/1410\nhttps://github.com/paradedb/paradedb/issues/1289\n'

## installing PostgreSQL

In [2]:
! sudo apt install gnupg2 wget nano
! sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
! curl -fsSL https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/postgresql.gpg
! sudo apt update
! sudo apt install postgresql-16 postgresql-contrib-16 postgresql-server-dev-16
! service postgresql start
! sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
gnupg2 is already the newest version (2.2.27-3ubuntu2.1).
Suggested packages:
  hunspell
The following NEW packages will be installed:
  nano
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 280 kB of archives.
After this operation, 881 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 nano amd64 6.2-1ubuntu0.1 [280 kB]
Fetched 280 kB in 1s (513 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-p

## installing ParadeDB pg_search

In [3]:
! export PARADEDB_TELEMETRY=false
! lsb_release -a
! sudo apt-get install -y libicu70

# https://github.com/paradedb/paradedb/releases
! wget https://github.com/paradedb/paradedb/releases/download/v0.10.2/postgresql-16-pg-search_0.10.2-1PARADEDB-jammy_amd64.deb
! dpkg -i postgresql-16-pg-search_0.10.2-1PARADEDB-jammy_amd64.deb

# https://docs.paradedb.com/deploy/pg_search
# https://github.com/paradedb/paradedb/tree/dev/pg_search

! echo "shared_preload_libraries = 'pg_search'" >> /etc/postgresql/16/main/postgresql.conf
! service postgresql stop
! sleep 10
! service postgresql start
! sudo -u postgres psql -c "CREATE EXTENSION pg_search;"

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libicu70 is already the newest version (70.1-2).
libicu70 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
--2024-11-04 09:23:34--  https://github.com/paradedb/paradedb/releases/download/v0.10.2/postgresql-16-pg-search_0.10.2-1PARADEDB-jammy_amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/660776503/7312ce8f-f2ad-41f4-a375-9578db0b479e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241104%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241104T092334Z&X-Amz-Expires=300&X-Amz-Signature=031c0cf5c43

## init psycopg2 helper

In [4]:
! pip install psycopg2

import psycopg2


def msq( t, verbose=False ) :
  res = []
  with psycopg2.connect("dbname=postgres user=root") as conn:
    with conn.cursor() as cur:
      try :
        cur.execute(t)
        res = cur.fetchall()
        if verbose :
          for cdesc in cur.description :
            print(cdesc[0],'|',end='')
          print('')
          for r in res:
            for c in r:
              print(c,'|',end='')
            print('')
      except Exception as ex :
        print('!!!! msq() ERROR ',ex,'|',t,'|')
  return res


def iscorrectintopk(correct, res) :
  cr = correct.replace('"','\'')
  if len(res) < 1 : return 0
  for cell in res[0] :
    if str(cell).replace('"','\'') == cr : return 2
  for ri in range(1,len(res)) :
    for cell in res[ri] :
      if str(cell).replace('"','\'') == cr : return 1
  return 0


## Test dataset: Wordpress related QA from Huggingface

In [5]:

! wget https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
! wget https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/queries.jsonl
! ls -la

import random, json

samplenum = 100

# load from jsonl
wcorpus = []
with open('corpus.jsonl') as f:
  wcstr = f.read()
  wcorpus = wcstr.split('\n')
print('len(wcorpus)',len(wcorpus))

# create sampled corpus, items, questions
sampledwcorpus = random.sample( wcorpus, samplenum )
items = []
qqs = []
for i in range(0,len(sampledwcorpus)) :
  wjs = json.loads(sampledwcorpus[i])
  #print(i,'---------------',wjs['_id'])
  #print(len(wjs['title']),wjs['title'])
  #print(len(wjs['text']),wjs['text'])
  items.append( { 'doctext': wjs['text'] } )
  qqs.append( [ wjs['title'], items[-1]['doctext'] ] )

# questions and solutions
random.shuffle(qqs)
questions = [ q[0] for q in qqs ]
questionsolutions = [ q[1] for q in qqs ]

# export to CSV for Postgres
csvfilename = 'items.csv'
with open(csvfilename,'w+') as f:
  f.write('id;doctext\n')
  for i in range(0,len(items)) :
    f.write(str((i+1))+';"'+items[i]['doctext'].replace('"','\'')+'"\n')


# Postgres creating items table by importing from CSV
msq('DROP TABLE IF EXISTS items CASCADE;')
msq('CREATE TABLE items (id SERIAL, doctext TEXT);')
msq('COPY items FROM \'/content/items.csv\' DELIMITER \';\' CSV HEADER;')
#msq('SELECT * from items;')


--2024-11-04 09:23:54--  https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
Resolving huggingface.co (huggingface.co)... 18.154.227.69, 18.154.227.7, 18.154.227.87, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/e3/a1/e3a12a6c68820b63bcd7ae09aa898026eba332004f38239070204b3b146060bc/089b11077372513eca8cc16653485aff1f232f8be18d8c6263be4b3b2bda0078?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27corpus.jsonl%3B+filename%3D%22corpus.jsonl%22%3B&Expires=1730971434&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDk3MTQzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2UzL2ExL2UzYTEyYTZjNjg4MjBiNjNiY2Q3YWUwOWFhODk4MDI2ZWJhMzMyMDA0ZjM4MjM5MDcwMjA0YjNiMTQ2MDYwYmMvMDg5YjExMDc3MzcyNTEzZWNhOGNjMTY2NTM0ODVhZmYxZjIzMmY4YmUxOGQ4YzYyNjNiZTRiM2IyYmRhMDA3OD9yZXNwb25zZS1j

[]

## plpgsql_bm25

In [6]:
# loading https://github.com/jankovicsandras/plpgsql_bm25

! wget https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql

# execute to load the functions
! psql postgresql://@/postgres -f /content/plpgsql_bm25.sql


--2024-11-04 09:23:57--  https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13696 (13K) [text/plain]
Saving to: ‘plpgsql_bm25.sql’

plpgsql_bm25.sql    100%[===================>]  13.38K  --.-KB/s    in 0s      

2024-11-04 09:23:57 (76.9 MB/s) - ‘plpgsql_bm25.sql’ saved [13696/13696]

psql:/content/plpgsql_bm25.sql:23: NOTICE:  function bm25simpletokenize() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:/content/plpgsql_bm25.sql:44: NOTICE:  function count_words_in_array() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:/content/plpgsql_bm25.sql:65: NOTICE:  function get_word_docs_count() does not exist, skipping
DROP FUNCTION
CREATE FUNCTIO

## BM25opt

In [7]:
! wget https://raw.githubusercontent.com/jankovicsandras/bm25opt/refs/heads/main/BM25opt.py

--2024-11-04 09:23:57--  https://raw.githubusercontent.com/jankovicsandras/bm25opt/refs/heads/main/BM25opt.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6279 (6.1K) [text/plain]
Saving to: ‘BM25opt.py’

BM25opt.py          100%[===================>]   6.13K  --.-KB/s    in 0s      

2024-11-04 09:23:57 (57.7 MB/s) - ‘BM25opt.py’ saved [6279/6279]



## running the comparsion of plpgsql_bm25, ParadeDB pg_search, rank_bm25, BM25opt
#### the order of results and the bm25 scores should be equal, but not the ```id```s
#### the ParadeDB parts are chaotic because unsuccessfully trying to avoid https://github.com/paradedb/paradedb/issues/1759

In [13]:

! pip install rank_bm25

from BM25opt import BM25opt, tokenizer_default
from rank_bm25 import BM25Okapi, BM25L, BM25Plus
import json, time, math, subprocess
import pandas as pd

# table and file names
tablename = 'items'
columnname = 'doctext'
k = 5
verbose = True
tablename_bm25wsmap = tablename+'_bm25wsmap'
csvfilepath = '/content/'+tablename_bm25wsmap+'.csv'
stats = {
  'plpgsql_bm25_correct':[],
  'paradedbbm25_correct':[],
  'rank_bm25_correct':[],
  'bm25opt_correct':[],
  'plpgsql_bm25_speed':[],
  'paradedbbm25_speed':[],
  'rank_bm25_speed':[],
  'bm25opt_speed':[],
}


# preparing tokenized corpus
tokenized_corpus = [ tokenizer_default(item[columnname]) for item in items ]

# rank_bm25 creating index
rank_bm25_index = BM25Okapi(tokenized_corpus)
# BM25opt creating index
bm25opt_index = BM25opt( [ item[columnname] for item in items ], algo='okapi' )

# Postgres has already "items" table, tokenization is built-in
msq( 'SELECT bm25createindex( \''+tablename+'\', \''+columnname+'\' );', verbose )

# ParadeDB pg_search
! psql postgresql://@/postgres -c "DROP SCHEMA IF EXISTS paradedbbm25 CASCADE;"
! psql postgresql://@/postgres -c "CALL paradedb.create_bm25( index_name => 'paradedbbm25', schema_name => 'public', table_name => 'items', key_field => 'id', text_fields => paradedb.field('doctext') );"
paradedbcharfilter = ['+','^','```', ':', '{', '}', '"', '[', ']', '(', ')', '~', '!', '\\', '\*', ' ']

def paradedbescape( s ):
  s2 = ''
  for c in s :
    if c in paradedbcharfilter : s2 += '\\'
    s2 += c
  return s2

def paradedbescape2( s ):
  wa = s.split()
  wa2 = []
  for w in wa :
    wa2.append(paradedbescape(w))
  return ' '.join(wa2).replace("'","\\\'")

def paradedbescape3( s ):
  wa = s.split()
  wa2 = []
  for w in wa :
    wa2.append('"'+paradedbescape(w)+'"')
  return ' '.join(wa2)

# Running the questions
for qi,q in enumerate(questions) :

  # tokenize and print question
  tokenizedquestion = tokenizer_default(q)
  if verbose :
    print('\n----Question',qi,':',q,' | Tokenized: ',tokenizedquestion)
    if questionsolutions and qi<len(questionsolutions) : print('Solution:',questionsolutions[qi])

  # plpgsql_bm25 BM25 search
  if verbose : print('plpgsql_bm25|')
  starttime = time.time()
  mres = msq( 'SELECT * FROM bm25topk( \''+tablename+'\', \''+columnname+'\', \''+q.replace("'","\'\'")+'\', '+str(k)+' );', verbose )
  stats['plpgsql_bm25_speed'].append( (time.time()-starttime) )
  stats['plpgsql_bm25_correct'].append( iscorrectintopk(questionsolutions[qi], mres) )


  # ParadeDB BM25 search
  pcmdstr = 'SELECT id, doctext FROM paradedbbm25.search( \''+columnname+':('+paradedbescape2( q )+')\', limit_rows => 5 );'
  #pcmdstr = 'SELECT id, doctext FROM paradedbbm25.search( '+columnname+':('+paradedbescape3( q )+'), limit_rows => 5 );'

  # executing query
  starttime = time.time()
  mres = msq( pcmdstr, verbose )
  stats['paradedbbm25_speed'].append( (time.time()-starttime) )
  stats['paradedbbm25_correct'].append( iscorrectintopk(questionsolutions[qi], mres) )

  # rank_bm25 BM25 search
  starttime = time.time()
  doc_scores = rank_bm25_index.get_scores( tokenizedquestion )
  stats['rank_bm25_speed'].append( (time.time()-starttime) )
  bres = [ [i,s] for i,s in enumerate(doc_scores) ]
  bres.sort(key=lambda x:x[1],reverse=True)
  bres = bres[:k]
  for i in range(0,len(bres)) :
    bres[i].append( items[ bres[i][0] ][columnname] )
    bres[i][0] += 1
  stats['rank_bm25_correct'].append( iscorrectintopk(questionsolutions[qi], bres) )
  if verbose :
    print('rank_bm25  |')
    for br in bres  : print( br[0], '|', br[1], '|', br[2] ) #, items[ br[0]-1 ][columnname] )

  # bm25opt BM25 search
  starttime = time.time()
  bres2 = bm25opt_index.topk( q, k )
  stats['bm25opt_speed'].append( (time.time()-starttime) )
  for i in range(0,len(bres2)) :
    bres2[i].append( items[ bres2[i][0] ][columnname] )
    bres2[i][0] += 1
  stats['bm25opt_correct'].append( iscorrectintopk(questionsolutions[qi], bres2) )
  if verbose :
    print('bm25opt   |')
    for br in bres2 : print( br[0], '|', br[1], '|', br[2] ) #items[ br[0]-1 ][columnname] )

  # print stats
  print(qi,'corrects',stats['plpgsql_bm25_correct'][-1],stats['paradedbbm25_correct'][-1],stats['rank_bm25_correct'][-1],stats['bm25opt_correct'][-1],'speeds',stats['plpgsql_bm25_speed'][-1],stats['paradedbbm25_speed'][-1],stats['rank_bm25_speed'][-1],stats['bm25opt_speed'][-1])


stats2 = {}
for statname in stats :
  stats2[statname+' total'] = [ sum(stats[statname]) ]

if verbose :
  df = pd.DataFrame(stats)
  #df.loc[:,'Total'] = df.sum(axis=1)
  df.style

df = pd.DataFrame(stats2)
df.style


bm25createindex |
 |
NOTICE:  drop cascades to 13 other objects
DETAIL:  drop cascades to index paradedbbm25_bm25_index
drop cascades to function paradedbbm25.search(text,integer,integer,text,boolean,text,text,text,integer,text,text,boolean,boolean)
drop cascades to function paradedbbm25.search(paradedb.searchqueryinput,integer,integer,text,boolean,text,text,text,integer,text,text,boolean,boolean)
drop cascades to function paradedbbm25.explain(text,integer,integer,text,boolean,text,text,text,integer,text,text,boolean,boolean)
drop cascades to function paradedbbm25.explain(paradedb.searchqueryinput,integer,integer,text,boolean,text,text,text,integer,text,text,boolean,boolean)
drop cascades to function paradedbbm25.schema()
drop cascades to function paradedbbm25.score_bm25(text,integer,integer,text,boolean,text,text,text,integer,text,text,boolean,boolean)
drop cascades to function paradedbbm25.score_bm25(paradedb.searchqueryinput,integer,integer,text,boolean,text,text,text,integer,text,t

,plpgsql_bm25_correct total,paradedbbm25_correct total,rank_bm25_correct total,bm25opt_correct total,plpgsql_bm25_speed total,paradedbbm25_speed total,rank_bm25_speed total,bm25opt_speed total
0,159,146,159,159,12.368801,8.830828,0.096890,0.019628
